In [2]:
from itertools import product
from collections import defaultdict
import math
from tqdm import tqdm
import re
import numpy as np

test, real = [], []
with open("./test.txt", "r+") as f:
    test = [l.rstrip() for l in f.readlines()]

with open("./real.txt", "r+") as f:
    real = [l.rstrip() for l in f.readlines()]

In [3]:
from collections import deque
def parse_line(line: str) -> tuple:
    lights_end = line.index(']')
    joltage_start = line.index('{')
    joltage_end = line.index('}')
    lights = [
        c == '#' for c in line[1: lights_end]
    ]
    buttons = []
    for text in line[lights_end + 1: joltage_start].split(' '):
        if len(text) == 0:
            continue
        buttons.append(list(map(int, text[1:-1].split(','))))
    
    joltages = list(map(
        int, 
        line[joltage_start + 1: joltage_end].split(',')
    ))

    return (lights, buttons, joltages)
    
def part1(lines: list[str]):
    s = 0
    for line in lines:
        lights, buttons, _ = parse_line(line)

        lights_ans = 0
        for idx, c in enumerate(lights):
            if c: lights_ans += (1 << idx)
        

        q = deque([(0, 0)])
        seen = set()
        while len(q) > 0:
            cur, steps = q.popleft()
            seen.add(cur)
            if cur == lights_ans:
                s += steps
                q.clear()
                continue
            for button in buttons:
                s_prime = cur
                for toggled in button:
                    i = (cur >> toggled) & 1
                    if i == 0:
                        s_prime += (1 << toggled)
                    else:
                        s_prime -= (1 << toggled)
                if s_prime not in seen:
                    q.append((s_prime, steps + 1))
            
        
    return s
print("TEST: ", part1(test))
print("REAL: ", part1(real))

TEST:  7
REAL:  447


In [ ]:

from scipy.optimize import LinearConstraint, milp
def part2(lines: list[str]):
    s = 0

    for line in lines:
        _, buttons, joltage = parse_line(line)
        b = np.array(joltage)
        A = np.array([
            [1 if i in button else 0 for i in range(len(joltage))] 
            for button in buttons
        ]).T

        res = milp(
            np.ones((len(buttons),)),
            constraints=LinearConstraint(A, b - 0.01, b + 0.01),
            integrality=1
        )
        s += np.sum(res.x)
    
    return s
        
print("TEST: ", part2(test))
print("REAL: ", part2(real))

TEST:  33.0
REAL:  18960.0
